# Работа с текстами

In [38]:
import pandas as pd
import numpy as np
import pickle

import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('ggplot')

In [32]:
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import ElasticNet

In [34]:
dff = pd.read_csv('cian_data_v1.csv', sep='\t')
y = dff["цена"].values
y.size

35109

In [35]:
f =  open('description_lemm.txt', "r")
T = f.readlines()
f.close()
len(T)

35109

## TF-IDF

In [37]:
T_train, T_test, y_train, y_test = train_test_split(T_lemm, y, test_size = 0.2)

grid_search_size=1000

vectorizer = TfidfVectorizer( ) # (min_df=10, max_df=0.8)#, max_features=4000)
regressor = ElasticNet()
    
pipe = Pipeline(steps=[('vectorizer', vectorizer),
                       ('regressor', regressor)])

parameters = dict(#vectorizer__min_df=np.arange(0,12,2),
#                   regressor__alpha=np.logspace(-4, 4, 50),
    regressor__alpha=np.logspace(-4, 4, 5)
#                   regressor__l1_ratio=[0, 0.1, 0.5, 0.7, 0.9, 0.95, 0.99, 1]
                 )

gridcv = GridSearchCV(pipe, parameters, scoring='neg_mean_absolute_error', cv=5, n_jobs=- 1)

if y_train.size <= grid_search_size:
    y_gs = y_train
    T_gs = T_train
else:
    T_gs, _, y_gs, _ = train_test_split(T_train, y_train, 
                                        test_size=(1 - grid_search_size/y_train.size), 
                                        random_state=42)
    
gridcv.fit(T_gs, y_gs)

print('Лучшее качество: {}'.format(gridcv.best_score_))
print('Лучшие параметры:', gridcv.best_params_, '\n')

model = gridcv.best_estimator_
model.fit(T_train, y_train)

Лучшее качество: -25127349.84829654
Лучшие параметры: {'regressor__alpha': 0.01} 



Pipeline(memory=None,
         steps=[('vectorizer',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, use_idf=True,
                                 vocabulary=None)),
                ('regressor',
                 ElasticNet(alpha=0.01, copy_X=True, fit_intercept=True,
                            l1_ratio=0.5, max_iter=1000, normal

In [41]:
np.mean(np.abs(model.predict(T_test) - y_test))/10**6

23.890105035907485

## W2V

In [50]:
from W2vVecorizer import W2vVecorizer

In [51]:
with open("w2v_models/w2v_Ivan/word2index.pickle", "rb") as f:
    word2index = pickle.load(f)
    
embedding = np.load("w2v_models/w2v_Ivan/embedding.npy")

vectorzer = W2vVecorizer(word2index, embedding)
vectorzer.fit(T_train)

In [54]:
vectorzer.nice_cnt/vectorzer.all_cnt 

0.985072603468474

In [52]:
T_vect = vectorzer.transform(T_train)
T_vect.shape

(28087, 300)